# Extract data from .sol files

Use this if you are using the Wilkens function mode 2 to extract <a> loop (phase 0) and <c> loop (phase 1)densities

This workbook is an example of how the extractDataDir function can be used to extract physical paramaters from a folder a .sol files. 

This data frame is then manipulated for plotting data from a synchrotron experiment as an example.

## Import stuff

In [202]:
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors
%matplotlib qt5

from src.cmwp_tools import extractDataDir
from src.xrd_tools import addLabels

## Extract data from solution files

This code extracts data from CMWP .sol files

In [203]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1
name='0.1_'

Parsing 104 solution files...
Parsed 103 solution files succesfully                                                                
"*** END" not present in 1                                                                     
Done!


In [158]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd_3p/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1
name='0.1_3p'

Parsing 117 solution files...
Parsed 117 solution files succesfully                                                                  
Done!


In [ ]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/hyd_3p_80_100/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [20.695, 21.195, 21.695]
edges = [14.918, 14.918, 14.919]
dpa_val = 0.1
name='0.1_3p_hor'

In [177]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa2/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [9.245, 9.745, 10.245, 10.745, 11.245]
edges = [14.839, 14.838, 14.835, 14.834, 14.832]
dpa_val = 2
name='2.0_'

Parsing 88 solution files...
Parsed 84 solution files succesfully                                                                 
"*** END" not present in 4                                                                     
Done!


In [169]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa05/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
#print(data_df['y'].unique())
unique = [-5.5, -5, -4.5, -4, -3.5]
edges = [14.780, 14.782, 14.782, 14.782, 14.782]
dpa_val = 0.5
name='0.5_'

Parsing 110 solution files...
Parsed 109 solution files succesfully                                                                 
"*** END" not present in 1                                                                     
Done!


In [173]:
directory = '/home/rhys/Documents/CMWP-211102/2020_11_DESY/dpa1/'
data_df = extractDataDir(directory+'*.sol', calcLoop=False, wavelength=0.018505104)
unique = [3.249, 3.749, 4.249]
edges = [14.820, 14.816, 14.812]
edges = [14.820, 14.818, 14.814]
dpa_val = 1
name='1.0_'

Parsing 78 solution files...
Parsed 76 solution files succesfully                                                                
"*** END" not present in 2                                                                     
Done!


## Pull out z and y values from filename, calculate depth from edge and add dpa

In [204]:
# Get rid of un-needed columns
data_df[['f2', 'y', 'f4', 'z']] = data_df['filename'].str.split('_',expand=True)               # Split by '_'
data_df.drop(['f2', 'f4'], axis=1, inplace=True)                        # Clear un-needed columns

# Format y and z as float
data_df['y'] = pd.to_numeric(data_df['y'], downcast="float")
data_df['z'] = pd.to_numeric(data_df['z'], downcast="float")

# Rearrange to put y and z first
cols = data_df.columns.tolist()
data_df=data_df[cols[-2:] + cols[:-2]]

# Calculate depth
data_df.insert(0, 'depth', data_df['y'])
data_df['depth'].replace(to_replace=unique, value=edges, inplace=True)
data_df['depth'] = data_df['z'] - data_df['depth']
data_df['depth'] = np.round(data_df['depth'] * 1000,1)

# Add dpa column
depth, dpa = np.loadtxt('data/srim.txt')
dpa *= dpa_val
depth_int = list(depth)+list((depth[0:-1]+depth[1:])/2)
dpa_int = list(dpa)+list((dpa[0:-1]+dpa[1:])/2)
index = np.argsort(depth_int)
depth_int = np.array(depth_int)[index]
dpa_int = np.array(dpa_int)[index]
data_df['dpa'] = [dpa_int[np.argmin(np.abs(i-depth_int))] for i in data_df['depth']]

phases = np.array(data_df['peak_phase'][3])
poss = np.array(data_df['peak_pos'][3])
names = np.array(data_df['peak_names'][3])

phaseNames = ['Zr', 'SPP', 'ZrH']
hkls = [names[phases==0], names[phases==2], names[phases==3]]
peakPos = [poss[phases==0], poss[phases==2], poss[phases==3]]
colours = ['b', 'r', 'g']

In [205]:
data_df

,depth,y,z,a1,a1_1,a2,a2_1,a_lat,a_lat_err,b,...,e_neg,e_pos,filename,peak_int,peak_names,peak_phase,peak_pos,rhoIGS,wssr,dpa
0,1.0,20.695,14.919,-0.5332,2.6411,-0.307,1.0058,3.237281,0.000195,364.943,...,0.035055,0.060198,y_20.695_z_14.919,"[453.834, 0.0, 3114.93, 0.0, 1801.75, 0.0, 470...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77944, 0.0, 4.10608, 0.0, 4.30165, 0.0, 5.5...",0.0,273.607,0.0577
1,3.0,20.695,14.921,-0.5332,2.6411,-0.307,1.0058,3.238030,0.000173,235.263,...,0.121793,0.164761,y_20.695_z_14.921,"[613.416, 0.0, 3615.17, 0.0, 2189.24, 0.0, 658...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77864, 0.0, 4.10613, 0.0, 4.3007, 0.0, 5.58...",0.0,555.002,0.0626
2,5.0,20.695,14.923,-0.5332,2.6411,-0.307,1.0058,3.237833,0.000165,252.502,...,0.058155,0.060684,y_20.695_z_14.923,"[601.963, 0.0, 3342.96, 0.0, 2015.57, 0.0, 621...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77894, 0.0, 4.10595, 0.0, 4.3006, 0.0, 5.58...",0.0,500.633,0.0694
3,7.0,20.695,14.925,-0.5332,2.6411,-0.307,1.0058,3.237687,0.000163,336.733,...,0.044942,0.058031,y_20.695_z_14.925,"[722.936, 0.0, 3276.79, 0.0, 1902.53, 0.0, 657...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77941, 0.0, 4.10579, 0.0, 4.30077, 0.0, 5.5...",0.0,427.219,0.0703
4,9.0,20.695,14.927,-0.5332,2.6411,-0.307,1.0058,3.237468,0.000166,267.692,...,0.033167,0.041900,y_20.695_z_14.927,"[658.676, 0.0, 3143.52, 0.0, 2037.61, 0.0, 683...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77939, 0.0, 4.10577, 0.0, 4.30124, 0.0, 5.5...",0.0,489.414,0.0783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,41.0,21.695,14.960,-0.5332,2.6411,-0.307,1.0058,3.237767,0.000180,368.222,...,30.950000,3.845000,y_21.695_z_14.960,"[662.087, 0.0, 3143.19, 0.0, 1904.29, 0.0, 587...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77966, 0.0, 4.10548, 0.0, 4.30178, 0.0, 5.5...",0.0,524.743,0.0000
99,43.0,21.695,14.962,-0.5332,2.6411,-0.307,1.0058,3.237762,0.000189,454.964,...,19.550000,0.000407,y_21.695_z_14.962,"[613.038, 0.0, 3348.12, 0.0, 2156.54, 0.0, 648...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77964, 0.0, 4.10544, 0.0, 4.30123, 0.0, 5.5...",0.0,572.849,0.0000
100,45.0,21.695,14.964,-0.5332,2.6411,-0.307,1.0058,3.237660,0.000184,317.553,...,19.800000,0.000432,y_21.695_z_14.964,"[590.624, 0.0, 3345.24, 0.0, 1973.24, 0.0, 702...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77965, 0.0, 4.10525, 0.0, 4.30137, 0.0, 5.5...",0.0,663.388,0.0000
101,47.0,21.695,14.966,-0.5332,2.6411,-0.307,1.0058,3.237662,0.000164,406.918,...,22.851833,11.795898,y_21.695_z_14.966,"[674.558, 0.0, 3449.77, 0.0, 1948.26, 0.0, 726...","[100, 100, 002, 002, 101, 101, 102, 102, 110, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...","[3.77985, 0.0, 4.10553, 0.0, 4.30145, 0.0, 5.5...",0.0,635.361,0.0000


## Plot every other

In [206]:
fnames = list(directory + data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['filename'].str[0:]+'.dat')[::2][:9]
depths = data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['depth'][::2][:9]
dpas = data_df[(data_df['y'] == unique[2]) & (data_df['depth'] > 2)]['dpa'][::2][:9]

from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')

colors = [cmap(i) for i in np.linspace(0, 0.9, len(fnames))]  # Linear colour against depth
colors = [cmap(i/np.max(dpas)) for i in dpas]                 # Linear colour against dpa

f, (lax, ax) = plt.subplots(2,1, gridspec_kw={'height_ratios': [1, 5]}, figsize = (10, 5), sharex=True)

i=0
for fname, depth, c, dpa in zip(fnames, depths, colors, dpas):
    x, y = np.loadtxt(fname, unpack=True)
    ax.plot(x,np.log(y)+i, 'o-', markersize=1,  color=c, lw=1, label=str(depth)[:-2] + 'um = ' + str(dpa)[:4] + ' dpa')
    i-=0.2
    
plt.legend()
#plt.yscale('log')
plt.xlim(np.min(x), np.max(x))

plt.xlabel('2theta (degree)')
plt.ylabel('log(Intensity)')


# plot labels
addLabels(ax=ax, lax=lax, names=phaseNames, hkls=hkls, positions=peakPos, colours=colours)

## Plot raw data

This makes a function so that when you click on data points in the next plot, it plots the raw data

In [207]:
def plotfig(num):
    
    f, (lax, ax) = plt.subplots(2,1, gridspec_kw={'height_ratios': [1, 5]}, figsize = (14, 8), sharex=True)
   
    data = list(directory + data_df['filename']+'.dat')[num]
    back = list(directory + data_df['filename']+'.int.bg.dat')[num]
    
    calc = list(directory + data_df['filename']+'.int.th.dat')[num]

    # Plot data
    x_data, y_data = np.loadtxt(data, unpack=True)
    ax.plot(x_data, y_data, '--', lw=1, c='g', label='measured')
    
    # Plot calculated
    x, y = np.loadtxt(calc, unpack=True)
    ax.plot(x, y, lw=1, c='r', label='calculated')

    # Plot background
    x, y = np.loadtxt(back, unpack=True)
    ax.plot(x, y, '--', c='b', lw=1, label='background')

    plt.legend()
    plt.yscale('log')
    plt.xlim(np.min(x), np.max(x))
    
    plt.xlabel('2theta (degrees)')
    plt.ylabel('Intensity')

    
    # plot labels
    
    addLabels(ax=ax, lax=lax, names=phaseNames, hkls=hkls, positions=peakPos, colours=colours)

## Plot

This plots dislocation density, <a> loop fraction, M paramater and lattice paramaters as a function of depth from sample edge

In [6]:
#data_df.to_csv('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Data_for_Andrew/Mar2022/' + name + 'data.csv')

In [208]:
fig, axes = plt.subplots(3, 3, figsize=(16,12))
fig.subplots_adjust(left=0.06, right=0.96, top=0.98, bottom=0.1, hspace=0.3, wspace = 0.35)
axes = axes.ravel()

param = 'Re'

max_x = 40

# Plot the dpa
for sax in axes:
    sax.set_xlabel('Depth [${\mu}m$]', fontsize=12)
    sec = sax.twinx()
    sec.fill(depth, np.array(dpa), alpha=0.2)
    sec.tick_params(axis='both', which='major', labelsize=10)
    sec.set_xlim(0, max_x)
    sec.set_ylim(0)
    sec.set_ylabel('Dose [dpa]', fontsize=12)
    
    sax.set_zorder(sec.get_zorder()+1)
    sax.patch.set_visible(False);

# Split the lattice param axis
lim_defs=[np.round(np.min(data_df['a_lat'])-0.002,3), np.round(np.max(data_df['a_lat'])+0.002,3), 
          np.round(np.min(data_df['c_lat'])-0.002,3), np.round(np.max(data_df['c_lat'])+0.002,3)]

a_ratio = (lim_defs[1]-lim_defs[0])/((lim_defs[3]-lim_defs[2])+(lim_defs[1]-lim_defs[0]))

pos_orig = axes[6].get_position() # get the original position 
axes = np.append(axes, plt.axes([pos_orig.x0, pos_orig.y0 + (pos_orig.height * (1-a_ratio)),  pos_orig.width, pos_orig.height * a_ratio] ))
axes[9].xaxis.set_ticklabels([])
axes[6].set_position([pos_orig.x0, pos_orig.y0,  pos_orig.width, pos_orig.height * (1-a_ratio)])

# Common settings for all axes
for sax in axes:
    sax.set_xlim(0, max_x)
    sax.tick_params(axis='both', which='major', labelsize=10)
    sax.xaxis.set_ticks_position('both')

#### dislocation density

axes[0].errorbar(x=data_df['depth'], y=data_df['d'], yerr = [data_df['d_neg'],data_df['d_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[0].set_ylabel('<a> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[0].set_ylim(0)

d_lim = data_df['d']>0.5
d_lim = data_df['depth']<30
d_1_lim = data_df['d_1']>0.5

if param == 'M':
    #### m



    axes[1].errorbar(x=data_df['depth'][d_lim], y=np.exp(2)*data_df['e'][d_lim], 
                     yerr = np.exp(2)*np.array([data_df['e_neg'][d_lim],data_df['e_pos'][d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)

    axes[1].set_ylabel('$M_{<a>}$', fontsize=12)
    axes[1].set_ylim(0)

    #### m
    axes[4].errorbar(x=data_df['depth'][d_1_lim], y=np.exp(2)*data_df['e_1'][d_1_lim], 
                     yerr = np.exp(2)*np.array([data_df['e_1_neg'][d_1_lim], data_df['e_1_pos'][d_1_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
    axes[4].set_ylabel('$M_{<c>}$', fontsize=12)
    axes[4].set_ylim(0)

if param == 'Re':

    axes[1].errorbar(x=data_df['depth'][d_lim], y=(100*data_df['e'][d_lim])/data_df['d'][d_lim], 
                     yerr = 100*np.array(
                         [data_df['e_neg'][d_lim]/data_df['d_neg'][d_lim],
                          data_df['e_pos'][d_lim]/data_df['d_pos'][d_lim]]
                     ), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)

    axes[1].set_ylabel('$Re*_{<a>}$', fontsize=12)
    axes[1].set_ylim(0)

    #### m
    axes[4].errorbar(x=data_df['depth'][d_1_lim], y=(100*data_df['e_1'][d_1_lim])/data_df['d_1'][d_1_lim], 
                     yerr = 100*np.array(
                         [data_df['e_1_neg'][d_1_lim]/data_df['d_1_neg'][d_1_lim],
                          data_df['e_1_pos'][d_1_lim]/data_df['d_1_pos'][d_1_lim]]
                     ), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)

    axes[4].set_ylabel('$Re*_{<c>}$', fontsize=12)
    axes[4].set_ylim(0)

    
    
    
#### dislocation density

axes[3].errorbar(x=data_df['depth'], y=data_df['d_1'], yerr = [data_df['d_1_neg'],data_df['d_1_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5)
axes[3].set_ylabel('<c> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[3].set_ylim(0)

#### a loop fraction
data_df['a_frac'] = data_df['d']/(data_df['d']+data_df['d_1'])
data_df.plot(ax=axes[2], x='depth', y='a_frac', ms=3, marker='o', c='k', lw=0,  legend=False, xlabel='Depth [${\mu}m$]', ylabel='$f_{<a>}$', ylim=(0,1))

#### c loop fraction
data_df['c_frac'] = data_df['d_1']/(data_df['d']+data_df['d_1'])
data_df.plot(ax=axes[5], x='depth', y='c_frac', ms=3, marker='o', c='k', lw=0,  legend=False, xlabel='Depth [${\mu}m$]', ylabel='$f_{<c>}$', ylim=(0,1))

#### c paramater
data_df.plot(ax=axes[6], x='depth', y='c_lat', yerr='c_lat_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='c [Å]', ylim=(lim_defs[2], lim_defs[3]))
axes[6].set_yticks(np.arange(lim_defs[2], lim_defs[3], 0.001)[:-1])

#### a paramater
data_df.plot(ax=axes[9], x='depth', y='a_lat', yerr='a_lat_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='', ylabel='a [Å]', ylim=(lim_defs[0], lim_defs[1]))
axes[9].set_yticks(np.arange(lim_defs[0], lim_defs[1], 0.001))

#### c/a
data_df.plot(ax=axes[7], x='depth', y='covera', yerr='covera_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='c/a ratio')

#### cell volume
data_df.plot(ax=axes[8], x='depth', y='covera', yerr='covera_err', ms=3, marker='o', c='k', lw=0, capsize=5, elinewidth=1, legend=False, xlabel='Depth [${\mu}m$]', ylabel='Cell volume [$Å^3$]')

plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/'+name+'_'+param+'_ellip1.pdf', dpi=600)
plt.show()

for ax in axes:
    crs = mplcursors.cursor(ax,hover=False)

    crs.connect("add", lambda sel: sel.annotation.set_text('y={:.3f}, z={:.3f}'.format(data_df['y'][int(sel.index)], data_df['z'][int(sel.index)])))
    crs.connect("add", lambda sel: plotfig(int(sel.index)))
    plt.show()
    

## Plot dpa vs a-loop dd

In [210]:
ignorefirst = 5
ignoreafter = 25

mask = (data_df['depth']>ignorefirst) & (data_df['depth']<ignoreafter)

plt.errorbar(data_df['dpa'][mask], 
             data_df['d'][mask], 
             yerr=[data_df['d_neg'][mask], data_df['d_pos'][mask]],
             ms=3, marker='o', lw=0, 
             capsize=5, elinewidth=1,
             label=name[:3]+' dpa')

plt.xlabel('Dose (DPA)')
plt.ylabel('<a> dislocation density [$10^{14} m^{-2}$]')
plt.legend()
plt.tight_layout()

In [ ]:
#plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/dpavdd_a_5_25_log.pdf', dpi=600)

## Plot dpa vs c-loop dd

In [214]:
plt.subplots(1,1,figsize=(6.5, 3.2))

(<Figure size 650x320 with 1 Axes>, <AxesSubplot:>)

In [215]:
ignorefirst = 5
ignoreafter = 25

mask = (data_df['depth']>ignorefirst) & (data_df['depth']<ignoreafter)


plt.errorbar(data_df['dpa'][mask], 
             data_df['d_1'][mask], 
             yerr=[data_df['d_1_neg'][mask], data_df['d_1_pos'][mask]],
             ms=3, marker='o', lw=0, 
             capsize=5, elinewidth=1,
             label=name[:3]+' dpa')
#
plt.xlabel('Dose (DPA)')
plt.ylabel('<c> dislocation density [$10^{14} m^{-2}$]')
plt.tight_layout()

In [119]:
#plt.errorbar(x = [0.8, 3], y = [0.38, 0.95], label = '3D-LPA', ms=10, marker='x', color='k', lw=0)

In [122]:
plt.legend(loc='lower right', ncol=2)

In [123]:
#plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/dpavdd_c_5_25.pdf', dpi=600)

## Re vs dpa

In [ ]:
data_df['Re'] = 100*(data_df['e']/np.sqrt(data_df['d']))

data_df['Re_neg'] = 100/np.sqrt(data_df['d']) * data_df['e_neg'] + (50*data_df['e'])/np.power(data_df['d'], 3/2) * data_df['d_neg']
data_df['Re_pos'] = 100/np.sqrt(data_df['d']) * data_df['e_pos'] + (50*data_df['e'])/np.power(data_df['d'], 3/2) * data_df['d_pos']

In [ ]:
ignorefirst = 5
ignoreafter = 25

mask = (data_df['depth']>ignorefirst) & (data_df['depth']<ignoreafter)

plt.errorbar(data_df['dpa'][mask], 
             data_df['Re'][mask], 
             yerr= [data_df['Re_neg'][mask], data_df['Re_pos'][mask]],
             ms=3, marker='o', lw=0, 
             capsize=5, elinewidth=1,
             label=name[:3]+' dpa')

plt.xlabel('Dose (DPA)')
plt.ylabel('Re* <a>')
plt.legend()

In [ ]:
plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/dpa_vs_Re_a_5_25.pdf', dpi=600)

In [37]:
ignorefirst = 6
ignoreafter = 25

mask = (data_df['depth']>ignorefirst) & (data_df['depth']<ignoreafter)

plt.errorbar(data_df['dpa'][mask], 
             np.exp(2)*data_df['e'][mask], 
             yerr= [np.exp(2)*data_df['e_neg'][mask], np.exp(2)*data_df['e_pos'][mask]],
             ms=3, marker='o', lw=0, 
             capsize=5, elinewidth=1,
             label=name[:3]+' dpa')

plt.xlabel('Dose (DPA)')
plt.ylabel('<a> M paramater')
plt.legend()

## Plot a-dd, c-dd, M vs depth

In [228]:
fig, axes = plt.subplots(2, 1, figsize=(7,8))
#fig.subplots_adjust(left=0.06, right=0.96, top=0.98, bottom=0.1, hspace=0.3, wspace = 0.35)
axes = axes.ravel()


max_x = 40

# Common settings for all axes
for sax in axes:
    sax.set_xlim(0, max_x)
    sax.tick_params(axis='both', which='major', labelsize=10)
    sax.xaxis.set_ticks_position('both')
    sax.set_xlabel('Depth [${\mu}m$]', fontsize=12)

#### dislocation density

axes[0].errorbar(x=data_df['depth'], y=data_df['d'], yerr = [data_df['d_neg'],data_df['d_pos']],  marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5, label = str(dpa_val) + ' dpa')
axes[0].set_ylabel('<a> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[0].set_ylim(0)
axes[0].legend()

#### m

d_lim = data_df['d']>0.5
d_lim = data_df['depth']<30
d_1_lim = data_df['d_1']>0.5

axes[1].errorbar(x=data_df['depth'][d_lim], y=np.exp(2)*data_df['e'][d_lim], 
                 yerr = np.exp(2)*np.array([data_df['e_neg'][d_lim],data_df['e_pos'][d_lim]]), marker='o',c='k',  ms=3, lw=0, elinewidth=1, capsize=5, label = str(dpa_val) + ' dpa')

axes[1].set_ylabel('$M_{<a>}$', fontsize=12)
axes[1].set_ylim(0)
axes[1].legend()

## plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/'+name+'_ellip1.pdf', dpi=600)
plt.show()

for ax in axes:
    crs = mplcursors.cursor(ax,hover=False)

    crs.connect("add", lambda sel: sel.annotation.set_text('y={:.3f}, z={:.3f}'.format(data_df['y'][int(sel.index)], data_df['z'][int(sel.index)])))
    crs.connect("add", lambda sel: plotfig(int(sel.index)))
    plt.show()
    

## Plot depth vs a-dd, c-dd, M

In [224]:
data_df = data_df.groupby(by=['depth'], as_index=False).mean()

In [225]:
fig, axes = plt.subplots(2, 2, figsize=(13, 6.4))
#fig.subplots_adjust(left=0.06, right=0.96, top=0.98, bottom=0.1, hspace=0.3, wspace = 0.35)
axes = axes.ravel()
max_x = 40

In [226]:
# Common settings for all axes
for sax in axes:
    sax.set_xlim(0, max_x)
    sax.tick_params(axis='both', which='major', labelsize=10)
    sax.xaxis.set_ticks_position('both')
    sax.set_xlabel('Depth [${\mu}m$]', fontsize=12)

#### dislocation density

axes[0].plot(data_df['depth'], data_df['d'], '-', lw=1, label = str(dpa_val) + ' dpa')
axes[0].set_ylabel('<a> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[0].set_ylim(0)
axes[0].legend()

#### m

d_lim = data_df['d']>0.5
d_lim = data_df['depth']<30
d_1_lim = data_df['d_1']>0.5

axes[1].plot(data_df['depth'][d_lim], np.exp(2)*data_df['e'][d_lim], '-', lw=1, label = str(dpa_val) + ' dpa')

axes[1].set_ylabel('$M_{<a>}$', fontsize=12)
axes[1].set_ylim(0)
axes[1].legend()

#### m

d_lim = data_df['d']>0.5
d_lim = data_df['depth']<30
d_1_lim = data_df['d_1']>0.5

axes[2].plot(data_df['depth'][d_lim], np.exp(2)*data_df['e_1'][d_lim], '-', lw=1, label = str(dpa_val) + ' dpa')

axes[2].set_ylabel('$M_{<c>}$', fontsize=12)
axes[2].set_ylim(0)
axes[2].legend()

#### dislocation density

axes[3].plot(data_df['depth'], data_df['d_1'], '-', lw=1, label = str(dpa_val) + ' dpa')
axes[3].set_ylabel('<c> dislocation density [$10^{14} m^{-2}$]', fontsize=12)
axes[3].set_ylim(0)
axes[3].legend()

plt.show()

for ax in axes:
    crs = mplcursors.cursor(ax,hover=False)

    crs.connect("add", lambda sel: sel.annotation.set_text('y={:.3f}, z={:.3f}'.format(data_df['y'][int(sel.index)], data_df['z'][int(sel.index)])))
    crs.connect("add", lambda sel: plotfig(int(sel.index)))
    plt.show()

plt.tight_layout()

In [188]:
#plt.savefig('/home/rhys/Dropbox (Research Group)/Postdoc/DESY/Results_new/alldpa.pdf', dpi=600)